#  Extract OFAM3 data

Get the data and compute the monthly climatology for various GWLs

Works with the following

Ocean Acidification daily fields pH and aragonite saturation state

SST daily fields

Temperature monthly fields where all depths, select depths and bottom depth can be output.
for this case the code writes to the same file

NPP, MLD, and Eta_t monthly climatology now produced too.


In [1]:
import xarray as xr
from dask_jobqueue import PBSCluster
from dask.distributed import Client
import numpy as np
import matplotlib.pyplot as plt
import datetime
import flox
import flox.xarray
import pandas as pd
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter

from glob import glob


In [2]:
import dask
# Set configuration options
dask.config.set({
    'distributed.comm.timeouts.connect': '90s',  # Timeout for connecting to a worker
    'distributed.comm.timeouts.tcp': '90s',  # Timeout for TCP communications
})

client = Client() # Client(threads_per_worker=1) # why?
client


2024-07-19 15:14:14,244 - distributed.preloading - INFO - Creating preload: /g/data/hh5/public/apps/dask-optimiser/schedplugin.py
2024-07-19 15:14:14,247 - distributed.utils - INFO - Reload module schedplugin from .py file
2024-07-19 15:14:14,258 - distributed.preloading - INFO - Import preload module: /g/data/hh5/public/apps/dask-optimiser/schedplugin.py


Modifying workers


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /node/gadi-mmem-clx-0001.gadi.nci.org.au/62220/proxy/8787/status,
Dashboard: /node/gadi-mmem-clx-0001.gadi.nci.org.au/62220/proxy/8787/status,Workers: 48
Total threads: 48,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39005,Workers: 48
Dashboard: /node/gadi-mmem-clx-0001.gadi.nci.org.au/62220/proxy/8787/status,Total threads: 48
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:45649,Total threads: 1
Dashboard: /node/gadi-mmem-clx-0001.gadi.nci.org.au/62220/proxy/44943/status,Memory: 0 B
Nanny: tcp://127.0.0.1:36161,


In [3]:
# Function to calculate monthly climatology and convert to single precision
def monthly_climatology(ds, time_dim):
    """
    Calculate the monthly climatology for a given dataset.

    This function takes an xarray dataset and computes the monthly climatology
    by averaging data for each month over all years. The function adds a 'month'
    dimension to the dataset based on the provided time dimension.

    Parameters:
        ds (xarray.Dataset): The input dataset containing the time dimension.
        time_dim (str): The name of the time dimension in the dataset.

    Returns
        xarray.Dataset: Dataset with the mean values for each month across all years. 
        The data type of returned values is float32.
    """
    ds['month'] = ds[time_dim].dt.month
    climatology_cohorts = flox.xarray.xarray_reduce(
        ds,
        'month',
        func='mean',
        method='cohorts',
    )
    return climatology_cohorts.astype(np.float32)

In [4]:
# Set GWL periods
GWL_periods = {
    'current': ('1995-01-01', '2014-12-31'),
    'GW1p2': ('2001-01-01', '2020-12-31'),
    'GW1p5': ('2015-01-01', '2034-12-31'),
    'GW2p0': ('2030-01-01', '2049-12-31'),
    'GW3p0': ('2053-01-01', '2072-12-31'),
    'GW4p0': ('2074-01-01', '2093-12-31')
}

In [5]:
#Append some historical data to the future dataset to get the data for GW1.2
def filenames(dir1,dir2,filename):
    patternh = sorted(glob(dir1 + filename+'_*.nc'))  # ocean_mld_*.nc
    pattern1 = sorted(glob(dir1 + filename+'_200[0-5]*.nc'))
    pattern2 = sorted(glob(dir2 + filename+'_*.nc'))
    patternf=pattern1+pattern2
# patternh - historical files; patternf- future files with some historical files too
    return patternh, patternf

    
# Set GWL periods
GWL_periods = {
    'current': ('1995-01-01', '2014-12-31'),
    'GW1p2': ('2001-01-01', '2020-12-31') }


In [6]:
# Process and save climatology data
def process_climatology(ds, time_dim, start, end, variable, period):
    data = getattr(ds, variable).sel(**{time_dim: slice(start, end)}).persist() 
    clim = monthly_climatology(data, time_dim).persist()

    file_path = f'/g/data/ia39/ncra/ocean/{variable}_climatology_{period}.nc'
    
    # Save as netCDF
    clim.to_netcdf(file_path, compute=True)

In [7]:
%%time 
# Process OA fields 
# Directory paths for OA fields created
dir1 = '/scratch/xv83/rxm599/historical/'
dir2 = '/scratch/xv83/rxm599/future/'
dir0 = '/scratch/xv83/rxm599/pi/'
# his files2014- files2019 cover years 2000 2005
pattern1 = sorted(glob(dir1 + 'files201[4-9]*.nc'))
pattern2 = sorted(glob(dir2 + 'files*.nc'))
# Load datasets with chunking
dbgc1 = xr.open_mfdataset(dir1 + "files*.nc", parallel=True, chunks={'TIME41': 10})
dbgc2 = xr.open_mfdataset(pattern1 + pattern2, parallel=True, chunks={'TIME41': 10})
dbgc0 = xr.open_mfdataset(dir0 + "files*.nc", parallel=True, chunks={'TIME41': 10})
# Process current period
process_climatology(dbgc1, 'TIME41', *GWL_periods['current'], 'OAR', 'current')
process_climatology(dbgc1, 'TIME41', *GWL_periods['current'], 'PH', 'current')

# Process PI
process_climatology(dbgc0, 'TIME41', *GWL_periods['current'], 'OAR', 'PI')
process_climatology(dbgc0, 'TIME41', *GWL_periods['current'], 'PH', 'PI')

# Process future
for period, (start, end) in GWL_periods.items():
    if period == 'current':
        continue
    process_climatology(dbgc2, 'TIME41', start, end, 'OAR', period)
    process_climatology(dbgc2, 'TIME41', start, end, 'PH', period)

CPU times: user 7min 18s, sys: 2min 29s, total: 9min 47s
Wall time: 15min 56s


In [14]:
%%time
# Process SST fields 
# Directory paths for SST
dir1 = '/g/data/fp2/OFAM3/jra55_historical.1/surface/'
dir2 = '/g/data/fp2/OFAM3/jra55_rcp8p5/surface/'
p1,p2=filenames(dir1,dir2,'ocean_temp_sfc')

# Load datasets with chunking
dsst1 = xr.open_mfdataset(p1, parallel=True, chunks={'Time': 10})
dsst2 = xr.open_mfdataset(p2, parallel=True, chunks={'Time': 10})

# Process SST
for period, (start, end) in GWL_periods.items():
    if period == 'current':
        process_climatology(dsst1, 'Time', start, end, 'temp', period)
    else:
        process_climatology(dsst2, 'Time', start, end, 'temp', period)

CPU times: user 4min 2s, sys: 58.5 s, total: 5min 1s
Wall time: 15min 23s


In [13]:
%%time
# Process NPP fields 
# Directory paths for NPP 
dir1 = '/g/data/fp2/OFAM3/jra55_historical.1/bgc_monthly_ave/'
dir2 = '/g/data/fp2/OFAM3/jra55_rcp8p5/bgc_monthly_ave/'
p1,p2=filenames(dir1,dir2,'ocean_bgc_2d_mth')

# Load datasets with chunking
dsst1 = xr.open_mfdataset(p1, parallel=True, chunks={'Time': 10})
dsst2 = xr.open_mfdataset(p2, parallel=True, chunks={'Time': 10})

# Process NPP
for period, (start, end) in GWL_periods.items():
    if period == 'current':
        process_climatology(dsst1, 'Time', start, end, 'pprod_gross_2d', period)
    else:
        process_climatology(dsst2, 'Time', start, end, 'pprod_gross_2d', period)


CPU times: user 43.5 s, sys: 4.36 s, total: 47.8 s
Wall time: 59.2 s


In [9]:
%%time
# Process MLD fields 
# Directory paths  
dir1 = '/g/data/fp2/OFAM3/jra55_historical.1/monthly_ave/'
dir2 = '/g/data/fp2/OFAM3/jra55_rcp8p5/monthly_ave/'
p1,p2=filenames(dir1,dir2,'ocean_mld')
# Load datasets with chunking
dsst1 = xr.open_mfdataset( p1, parallel=True, chunks={'Time': 10})
dsst2 = xr.open_mfdataset( p2, parallel=True, chunks={'Time': 10})

# Process 
for period, (start, end) in GWL_periods.items():
    if period == 'current':
        process_climatology(dsst1, 'Time', start, end, 'mld_rho', period)
    else:
        process_climatology(dsst2, 'Time', start, end, 'mld_rho', period)


CPU times: user 60 s, sys: 9.87 s, total: 1min 9s
Wall time: 2min 18s


In [15]:
%%time
# Process ETA fields 
# Directory paths  
dir1 = '/g/data/fp2/OFAM3/jra55_historical.1/monthly_ave/'
dir2 = '/g/data/fp2/OFAM3/jra55_rcp8p5/monthly_ave/'
p1,p2=filenames(dir1,dir2,'ocean_month')
# Load datasets with chunking
dsst1 = xr.open_mfdataset(p1, parallel=True, chunks={'Time': 10})
dsst2 = xr.open_mfdataset(p2, 
                drop_variables=["temp_zflux_adv_full","temp_vdiffuse_impl_full","salt_zflux_adv_full","salt_vdiffuse_impl_full"], 
                          parallel=True, chunks={'Time': 10})

# Process 
for period, (start, end) in GWL_periods.items():
    if period == 'current':
        process_climatology(dsst1, 'Time', start, end, 'eta_t', period)
    else:
        process_climatology(dsst2, 'Time', start, end, 'eta_t', period)


CPU times: user 1min 4s, sys: 4.13 s, total: 1min 8s
Wall time: 5min 45s


In [13]:
%%time
# Process BMF_U and V fields 
# Directory paths  
dir1 = '/g/data/fp2/OFAM3/jra55_historical.1/forcing/'
dir2 = '/g/data/fp2/OFAM3/jra55_rcp8p5/forcing/'
p1,p2=filenames(dir1,dir2,'ocean_force_mth')
# Load datasets with chunking
dsst1 = xr.open_mfdataset(p1, parallel=True, chunks={'Time': 10})
dsst2 = xr.open_mfdataset(p2, 
#                drop_variables=["temp_zflux_adv_full","temp_vdiffuse_impl_full","salt_zflux_adv_full","salt_vdiffuse_impl_full"], 
                          parallel=True, chunks={'Time': 10})

# Process
for period, (start, end) in GWL_periods.items():
    if period == 'current':
        bmf=(dsst1.bmf_u**2+dsst1.bmf_v**2)**.5
        d1=bmf.to_dataset(name='bmf')
        process_climatology(d1, 'Time', start, end, 'bmf', period)
    else:
        bmf=(dsst2.bmf_u**2+dsst2.bmf_v**2)**.5
        d2=bmf.to_dataset(name='bmf')
        process_climatology(d2, 'Time', start, end, 'bmf', period)


CPU times: user 1min 27s, sys: 14.2 s, total: 1min 41s
Wall time: 1min 49s


## Process depth dependent variables 

In [38]:
#Append some historical data to the future dataset to get the data for GW1.2
def filenames1(dir1,dir2,filename):
    patternh = sorted(glob(dir1 + filename+'_*.nc'))  # ocean_mld_*.nc
    pattern1 = sorted(glob(dir1 + filename+'_200[0-5]*.nc'))
    pattern2 = sorted(glob(dir2 + filename+'_20*.nc'))
    patternf=pattern1+pattern2
# patternh - historical files; patternf- future files with some historical files too
    return patternh, patternf #pattern2

In [42]:
%%time
# Process TEMP fields 
# Directory paths for Temp
dir1 = '/g/data/fp2/OFAM3/jra55_historical.1/monthly_ave/'
dir2 = '/g/data/fp2/OFAM3/jra55_rcp8p5/monthly_ave/'
p1,p2=filenames1(dir1,dir2,'ocean_temp_mth')

# Load datasets with chunking
# needed to clean up future files to enable combining them with historical files
dsst1 = xr.open_mfdataset(p1 , parallel=True, chunks={'Time': 1 }) 
dsst2 = xr.open_mfdataset(p2, parallel=True, chunks={'Time': 1 },
        drop_variables=["st_edges_ocean","Time_bounds","average_DT",
            "average_T1","average_T2","nv"]  )

# define some useful variables
# select depth range 
da1=dsst1.sel(st_ocean=slice(0, 1000))
da2=dsst2.sel(st_ocean=slice(0, 1000))
# select a depth
#da1=dsst1.isel(st_ocean=(14))  # near 100m
#da2=dsst2.isel(st_ocean=(14))
# fill in depth to get the bottom value
db1=dsst1.ffill(dim="st_ocean", limit=None).isel(st_ocean=-1)
db2=dsst2.ffill(dim="st_ocean", limit=None).isel(st_ocean=-1)

CPU times: user 9.78 s, sys: 1.43 s, total: 11.2 s
Wall time: 11.2 s


In [46]:
%%time
# Can works on all depths (dsst1 dsst2)
# Reduced depths (da1 da2)
# Bottom depth (db1 db2)
# Process TEMP 
for period, (start, end) in GWL_periods.items():
    if period == 'current':
#        process_climatology(dsst1, 'Time', start, end, 'temp', period)
#        process_climatology(da1, 'Time', start, end, 'temp', period)
#        process_climatology(db1, 'Time', start, end, 'temp', period)
        print(period)
    else:
        process_climatology(dsst2, 'Time', start, end, 'temp', period)
#        process_climatology(da2, 'Time', start, end, 'temp', period)
#        process_climatology(db2, 'Time', start, end, 'temp', period)
        print(period)

# Need to move the files to correct name (bot_temp directory ) 

current
GW1p2
CPU times: user 5min 31s, sys: 2min 56s, total: 8min 27s
Wall time: 13min 13s


2024-07-19 16:05:16,010 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:38761'.
2024-07-19 16:05:16,012 - distributed.worker - ERROR - Scheduler was unaware of this worker 'tcp://127.0.0.1:38761'. Shutting down.
2024-07-19 16:05:16,404 - distributed.worker - ERROR - Failed to communicate with scheduler during heartbeat.
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.10/lib/python3.10/site-packages/distributed/comm/tcp.py", line 225, in read
    frames_nosplit_nbytes_bin = await stream.read_bytes(fmt_size)
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.10/lib/python3.10/site-packages/distributed/worker.py", line 1252, in heartbeat
    response = await retry_operation(
  File "/g/data/hh5/public/apps/miniconda3/envs/analy